In [ ]:
import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import hist
from hist import Hist

import time

import myPIDselector



In [ ]:
eps = myPIDselector.PIDselector("e")
pps = myPIDselector.PIDselector("p")
pips = myPIDselector.PIDselector("pi")
Kps = myPIDselector.PIDselector("K")
mups = myPIDselector.PIDselector("mu")


In [ ]:
print(len(pps.selectors))

pps.selectors

In [ ]:
pps.bits

In [ ]:
filename = 'SP-1005-LambdaVeryVeryLoose-Run3-R24a2-v03_COMBINED.root'

f = uproot.open(filename)

# This extracts the ROOT ntuple (TTree)
# From this we can extract awkward arrays
t = f['ntp1']

print(type(t))

In [ ]:
n = 1

pe = t['penergy'].array()[n]
trke = t['TRKenergy'].array()[n]

pbit = t['pSelectorsMap'].array()[n]
ptrkidx = t['pTrkIdx'].array()[n]


print(pbit)
print(trke)
print()
print(pe)
print(ptrkidx)
print()

#pps.SetBits(pbit)

#binrep = np.binary_repr(pbit)

#? np.binary_repr

binary_repr_vec = np.vectorize(np.binary_repr)
binrep = binary_repr_vec(pbit, width=32)

print(binrep)
print()
#tempbits = np.array(list(binrep[::-1])).astype(int)
tempbits = np.array(binrep).astype(int)

print(tempbits)
print()


pps.SetBits(pbit[0])
print(pps.IsBitSet(15))
print(pps.IsSelectorSet("LooseKMProtonSelection"))
print(pps.IsSelectorSet("TightKMProtonSelection"))
print(pps.IsSelectorSet("VeryTightKMProtonSelection"))
print(pps.IsSelectorSet("SuperTightKMProtonSelection"))

In [ ]:
# Trying to get ak working

n = 1

pe = t['penergy'].array()[n]
trke = t['TRKenergy'].array()[n]

pbit = t['pSelectorsMap'].array()[n]
ptrkidx = t['pTrkIdx'].array()[n]


print(pbit)
print(trke)
print()
print(pe)
print(ptrkidx)
print()

x = pbit[ptrkidx]
print("pbit[ptrkidx]")
print(x)
print()

#pps.SetBits(pbit)

#binrep = np.binary_repr(pbit)

#? np.binary_repr

binary_repr_vec = np.vectorize(np.binary_repr)
binrep = binary_repr_vec(x, width=32)

print(binrep)
print()
#tempbits = np.array(list(binrep[::-1])).astype(int)
tempbits = np.array(binrep).astype(int)

print(tempbits)
print()


#pps.SetBits(pbit[0])
#print(pps.IsBitSet(15))
#print(pps.IsSelectorSet("LooseKMProtonSelection"))
#print(pps.IsSelectorSet("TightKMProtonSelection"))
#print(pps.IsSelectorSet("VeryTightKMProtonSelection"))
#print(pps.IsSelectorSet("SuperTightKMProtonSelection"))

In [ ]:
# Trying to get ak working

# Loops are slow!

allbits = []

#nevents = len(t['penergy'].array())
nevents = 10

for n in range(nevents):
    #n = 1

    if n%10==0:
        print(n)
        
    pe = t['penergy'].array()[n]
    trke = t['TRKenergy'].array()[n]
    
    pbit = t['pSelectorsMap'].array()[n]
    ptrkidx = t['pTrkIdx'].array()[n]
    
    
    #print(pbit)
    #print(trke)
    #print()
    #print(pe)
    #print(ptrkidx)
    #print()
    
    x = pbit[ptrkidx]
    #print("pbit[ptrkidx]")
    #print(x)
    #print()
    
    #pps.SetBits(pbit)
    
    #binrep = np.binary_repr(pbit)
    
    #? np.binary_repr
    
    binary_repr_vec = np.vectorize(np.binary_repr)
    binrep = binary_repr_vec(x, width=32)
    
    #print(binrep)
    #print()
    #tempbits = np.array(list(binrep[::-1])).astype(int)
    tempbits = np.array(binrep).astype(int)
    
    #print(tempbits)
    #print()

    allbits.append(tempbits)

allbits = ak.Array(allbits)

allbits

#pps.SetBits(pbit[0])
#print(pps.IsBitSet(15))
#print(pps.IsSelectorSet("LooseKMProtonSelection"))
#print(pps.IsSelectorSet("TightKMProtonSelection"))
#print(pps.IsSelectorSet("VeryTightKMProtonSelection"))
#print(pps.IsSelectorSet("SuperTightKMProtonSelection"))

In [ ]:
allbits[2]

In [ ]:
nevents = len(t['penergy'].array())
print(f"nevents: {nevents}")

n = nevents

pe = t['penergy'].array()[0:n]
trke = t['TRKenergy'].array()[0:n]

pbit = t['pSelectorsMap'].array()[0:n]
ptrkidx = t['pTrkIdx'].array()[0:n]


print(pbit)
print(trke)
print()
print(pe)
print(ptrkidx)
print()

binary_repr_vec = np.vectorize(np.binary_repr)

counts = ak.num(pbit)
binrep = binary_repr_vec(ak.flatten(pbit), width=32)

print(binrep)
print()
#tempbits = np.array(list(binrep[::-1])).astype(int)
tempbits = np.array(binrep).astype(int)

print(tempbits)
print()

newtempbits = ak.unflatten(tempbits,counts)
print(newtempbits)

'''
pps.SetBits(pbit[0])
print(pps.IsBitSet(15))
print(pps.IsSelectorSet("LooseKMProtonSelection"))
print(pps.IsSelectorSet("TightKMProtonSelection"))
print(pps.IsSelectorSet("VeryTightKMProtonSelection"))
print(pps.IsSelectorSet("SuperTightKMProtonSelection"))
'''
;

In [ ]:
n = 1

lamd1idx = t['Lambda0d1Idx'].array()[n]
lamd1lund = t['Lambda0d1Lund'].array()[n]
print(lamd1idx)
print(lamd1lund)
print()


pe = t['penergy'].array()[n]
trke = t['TRKenergy'].array()[n]

pbit = t['pSelectorsMap'].array()[n]
ptrkidx = t['pTrkIdx'].array()[n]


print(pbit)
print(trke)
print()
print(pe)
print(ptrkidx)
print()

binary_repr_vec = np.vectorize(np.binary_repr)
binrep = binary_repr_vec(pbit, width=32)

print(binrep)
print()
#tempbits = np.array(list(binrep[::-1])).astype(int)
tempbits = np.array(binrep).astype(int)

print(tempbits)
print()


pps.SetBits(pbit[0])
print(pps.IsBitSet(15))
print(pps.IsSelectorSet("LooseKMProtonSelection"))
print(pps.IsSelectorSet("TightKMProtonSelection"))
print(pps.IsSelectorSet("VeryTightKMProtonSelection"))
print(pps.IsSelectorSet("SuperTightKMProtonSelection"))